In [8]:
# !pip install pandas

In [9]:
import sqlite3
import random
#import pandas as pd

In command line: 

```
#launch sqlite command line tool & create database called resourcesDB
sqlite3 resourcesDB   

# tab-delimted file
.separator "\t"

# create new table called "eduresource" w/ the content from our tsv
.import edu_resource.csv eduresource
```

In [10]:
# Next up: connecting to our data
# online tutorial: https://www.sqlitetutorial.net/sqlite-python/sqlite-python-select/

In [11]:
# add .csv to sqlite table
import pandas as pd 
import sqlite3

conn = sqlite3.connect("./resourcesDB")
dtypes = {'DATEKEY': int, 'ID': str, 'TYPE': str, 'VALUE_TO_PAY': float, 'PAYMENT_DATE': str}
parseDates = ['PAYMENT_DATE']
data = pd.read_csv('../obligations.csv', sep=';', dtype=dtypes, parse_dates=parseDates)
data.to_sql("dataObligations", conn, if_exists='append', index=False)


In [12]:
# delete everything from data_obligations
import sqlite3

#sql = 'DELETE FROM dataObligations'
#conn = sqlite3.connect("resourcesDB")
#cur = conn.cursor()
#cur.execute(sql)
#conn.commit()

In [13]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)

    return conn
'''
def get_everything(conn):
    cursor = conn.execute('select * from dataObligations')
    names = list(map(lambda x: x[0], cursor.description))

    print(names)
'''

def select_by_slot(conn, slot_name, slot_value):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    cur.execute(f"""SELECT * FROM dataObligations
                WHERE {slot_name}='{slot_value}'""")

    rows = cur.fetchall()

    if len(list(rows)) < 1:
        print("There are no resources matching your query.")
    else:
        for row in random.sample(rows, 1):
            print(f"Try the {(row[4]).lower()} {row[0]} by {row[1]}. You can find it at {row[2]}.")

#get_everything(create_connection("./resourcesDB"))
select_by_slot(create_connection("./resourcesDB"),
    slot_name="ID",slot_value="VAT")

Try the 2021-05-20 00:00:00 202104 by VAT. You can find it at VAT.


In [14]:
# This files contains your custom actions which can be used to run
# custom Python code.
#
# See this guide on how to implement these action:
# https://rasa.com/docs/rasa/custom-actions

from typing import Any, Text, Dict, List

from rasa_sdk import Action, Tracker
from rasa_sdk.executor import CollectingDispatcher

import sqlite3
import random


class QueryResourceType(Action):

    def name(self) -> Text:
        return "query_resource_type"

    def run(self, dispatcher: CollectingDispatcher,
            tracker: Tracker,
            domain: Dict[Text, Any]) -> List[Dict[Text, Any]]:

        conn = create_connection(self, db_file="../primavera_db/resourcesDB")

        slot_value = tracker.get_slot("obligation_type")
        slot_name = "ID"

        get_query_results = select_by_slot(self, conn=conn,
            slot_name=slot_name,slot_value=slot_value)
        dispatcher.utter_message(text=get_query_results)

        return []

    def create_connection(db_file):
        """ create a database connection to the SQLite database
            specified by the db_file
        :param db_file: database file
        :return: Connection object or None
        """
        conn = None
        try:
            conn = sqlite3.connect(db_file)
        except Error as e:
            print(e)

        return conn

    def select_by_slot(conn, slot_name, slot_value):
        """
        Query all rows in the tasks table
        :param conn: the Connection object
        :return:
        """
        cur = conn.cursor()
        cur.execute(f"""SELECT * FROM eduresource
                    WHERE {slot_name}='{slot_value}'""")

        rows = cur.fetchall()

        if len(list(rows)) < 1:
            return[("There are no resources matching your query.")]
        else:
            for row in random.sample(rows, 1):
                return[print(f"Try the {(row[4]).lower()} {row[0]} by {row[1]}. You can find it at {row[2]}.")]

In [15]:
# from: https://techoverflow.net/2019/10/14/how-to-list-tables-in-sqlite3-database-in-python/
def tables_in_sqlite_db(conn):
    cursor = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = [
        v[0] for v in cursor.fetchall()
        if v[0] != "sqlite_sequence"
    ]
    cursor.close()
    return tables

In [16]:
conn = QueryResourceType.create_connection("../resourcesDB")
print(tables_in_sqlite_db(conn))

[]
